# 手工实现三层神经网络分类器项目说明

本笔记本详细介绍了一个从零开始手工实现的三层神经网络分类器项目。该项目不依赖深度学习框架，而是通过NumPy手动实现了神经网络的前向传播、反向传播、各种激活函数、损失函数和优化器等核心组件。

## 1. 项目概述

本项目实现了一个完整的三层神经网络分类器，包括：

- **模型架构**：输入层 → 隐藏层1 → 隐藏层2 → 输出层
- **激活函数**：支持多种激活函数（Sigmoid、ReLU、Tanh、Softmax）
- **损失函数**：交叉熵损失函数
- **优化器**：SGD优化器（支持动量和L2正则化）
- **学习率调度**：支持学习率衰减策略
- **超参数搜索**：支持调节学习率、隐藏层大小、正则化强度等超参数

项目的主要目标是通过手动实现神经网络的各个组件，深入理解神经网络的工作原理和训练过程。

## 2. 项目结构

项目由以下文件组成：

```
├── model.py          # 神经网络模型定义，包含前向传播和反向传播
├── activations.py    # 激活函数及其导数的实现
├── losses.py         # 损失函数（交叉熵）及其导数的实现
├── optimizer.py      # SGD优化器实现，包含学习率调度和L2正则化
├── train.py          # 训练逻辑，包含模型训练和验证
├── test.py           # 测试逻辑，用于评估模型性能
├── hyperparameter_search.py  # 超参数搜索
├── utils.py          # 工具函数，包括数据加载和预处理
├── main.py           # 主程序入口
└── config.py         # 配置文件，包含默认超参数
```

这种模块化的设计使得代码结构清晰，便于理解和扩展。

## 3. 神经网络模型

### 3.1 模型架构

本项目实现的是一个三层神经网络，包括：
- 输入层：接收原始特征
- 两个隐藏层：使用可配置的激活函数（默认为ReLU）
- 输出层：使用Softmax激活函数进行多分类

模型的参数包括：
- W1, b1：第一隐藏层的权重和偏置
- W2, b2：第二隐藏层的权重和偏置
- W3, b3：输出层的权重和偏置

### 3.2 前向传播

前向传播是神经网络计算输出的过程。对于输入数据X，前向传播的计算过程如下：

```python
# 第一隐藏层
z1 = np.dot(X, W1) + b1
a1 = activation_function(z1)

# 第二隐藏层
z2 = np.dot(a1, W2) + b2
a2 = activation_function(z2)

# 输出层
z3 = np.dot(a2, W3) + b3
a3 = softmax(z3)
```

其中：
- X是输入数据，形状为(batch_size, input_size)
- W1, W2, W3是权重矩阵
- b1, b2, b3是偏置向量
- z1, z2, z3是线性变换的结果
- a1, a2, a3是激活函数的输出
- activation_function是隐藏层的激活函数（如ReLU）
- softmax是输出层的激活函数

### 3.3 反向传播

反向传播是计算损失函数对模型参数的梯度的过程，用于更新模型参数。反向传播的计算过程如下：

1. 计算输出层的误差：
   ```python
   # dout是损失函数对输出层激活值的梯度
   dz3 = dout * output_activation.backward(z3)
   ```

2. 计算输出层参数的梯度：
   ```python
   dW3 = np.dot(a2.T, dz3)
   db3 = np.sum(dz3, axis=0)
   ```

3. 计算第二隐藏层的误差：
   ```python
   da2 = np.dot(dz3, W3.T)
   dz2 = da2 * hidden_activation.backward(z2)
   ```

4. 计算第二隐藏层参数的梯度：
   ```python
   dW2 = np.dot(a1.T, dz2)
   db2 = np.sum(dz2, axis=0)
   ```

5. 计算第一隐藏层的误差：
   ```python
   da1 = np.dot(dz2, W2.T)
   dz1 = da1 * hidden_activation.backward(z1)
   ```

6. 计算第一隐藏层参数的梯度：
   ```python
   dW1 = np.dot(X.T, dz1)
   db1 = np.sum(dz1, axis=0)
   ```

通过这种链式求导的方式，我们可以计算出损失函数对所有参数的梯度，然后使用优化器更新参数。

## 4. 激活函数

本项目实现了多种激活函数，每种激活函数都包含前向传播和反向传播（计算导数）的方法。

### 4.1 Sigmoid

Sigmoid函数将输入映射到(0,1)区间，常用于二分类问题。

- 函数定义：$f(x) = \frac{1}{1 + e^{-x}}$
- 导数：$f'(x) = f(x) \cdot (1 - f(x))$

### 4.2 ReLU (Rectified Linear Unit)

ReLU是目前最常用的激活函数之一，它在正数部分是线性的，在负数部分为零。

- 函数定义：$f(x) = \max(0, x)$
- 导数：$f'(x) = \begin{cases} 1 & \text{if } x > 0 \\ 0 & \text{if } x \leq 0 \end{cases}$

### 4.3 Tanh

Tanh函数将输入映射到(-1,1)区间，输出以0为中心。

- 函数定义：$f(x) = 	anh(x)$
- 导数：$f'(x) = 1 - f(x)^2$

### 4.4 Softmax

Softmax函数将输入转换为概率分布，常用于多分类问题的输出层。

- 函数定义：$f(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}$
- 注意：Softmax的导数通常与交叉熵损失函数一起计算，以简化计算

## 5. 损失函数

本项目实现了交叉熵损失函数，它是多分类问题中最常用的损失函数。

### 5.1 交叉熵损失函数

交叉熵损失函数衡量预测概率分布与真实标签分布之间的差异。

- 函数定义：$L = -\sum_{i} y_{true,i} \log(y_{pred,i})$
- 导数（当与Softmax结合使用时）：$\frac{\partial L}{\partial z_i} = y_{pred,i} - y_{true,i}$

其中：
- $y_{true,i}$是真实标签（通常是one-hot编码）
- $y_{pred,i}$是模型预测的概率
- $z_i$是Softmax层的输入

## 6. 优化器

本项目实现了随机梯度下降（SGD）优化器，并支持动量和L2正则化。

### 6.1 SGD优化器

SGD是最基本的优化算法，通过梯度下降来更新模型参数。

- 基本更新规则：$	heta = 	heta - \alpha \cdot 
abla J(	heta)$

其中：
- $	heta$是模型参数
- $\alpha$是学习率
- $
abla J(	heta)$是损失函数对参数的梯度

### 6.2 动量

动量是对SGD的改进，它通过累积过去的梯度来加速收敛并减少震荡。

- 速度更新：$v = \beta \cdot v - \alpha \cdot 
abla J(	heta)$
- 参数更新：$	heta = 	heta + v$

其中：
- $v$是速度（累积梯度）
- $\beta$是动量系数（通常为0.9）

### 6.3 L2正则化

L2正则化通过在损失函数中添加权重的平方和来防止过拟合。

- 正则化项：$\lambda \sum_{i} 	heta_i^2$
- 梯度更新：$
abla J(	heta) = 
abla J(	heta) + 2\lambda	heta$

其中：
- $\lambda$是正则化强度

### 6.4 学习率调度

学习率调度是根据训练进度动态调整学习率的技术。本项目实现了几种常见的学习率调度策略：

- **固定学习率**：整个训练过程使用相同的学习率
- **步进衰减**：每隔一定的epoch，学习率乘以一个衰减因子
- **指数衰减**：学习率按指数函数衰减
- **余弦退火**：学习率按余弦函数从初始值衰减到最小值

## 7. 训练过程

训练神经网络的过程包括以下步骤：

1. **数据准备**：
   - 加载数据集（如CIFAR-10）
   - 数据预处理（标准化、归一化等）
   - 划分训练集和验证集

2. **模型初始化**：
   - 创建神经网络模型
   - 初始化模型参数（使用Xavier/Glorot初始化）
   - 设置损失函数、优化器和学习率调度器

3. **训练循环**：
   - 对于每个epoch：
     - 将训练数据分成小批量
     - 对于每个小批量：
       - 前向传播计算预测值
       - 计算损失
       - 反向传播计算梯度
       - 更新模型参数
     - 在验证集上评估模型
     - 保存最佳模型
     - 更新学习率（如果使用学习率调度）

4. **模型保存**：
   - 保存训练好的模型参数
   - 记录训练历史（损失、准确率等）

## 8. 超参数搜索

超参数搜索是寻找最佳超参数组合的过程。本项目支持调节以下超参数：

- **学习率**：控制参数更新的步长
- **隐藏层大小**：第一和第二隐藏层的神经元数量
- **批量大小**：每次更新使用的样本数量
- **动量系数**：控制动量的强度
- **正则化强度**：控制L2正则化的强度
- **激活函数**：隐藏层使用的激活函数类型
- **学习率调度策略**：学习率随时间变化的方式

超参数搜索的步骤：

1. 定义超参数搜索空间
2. 对于每种超参数组合：
   - 创建并训练模型
   - 在验证集上评估模型性能
   - 记录结果
3. 选择性能最佳的超参数组合

## 9. 使用方法

本项目提供了简单的命令行接口，可以通过以下方式使用：

### 9.1 训练模型

```bash
python main.py --mode train
```

可选参数：
- `--learning_rate`：学习率
- `--hidden_size1`：第一隐藏层大小
- `--hidden_size2`：第二隐藏层大小
- `--batch_size`：批量大小
- `--epochs`：训练轮数
- `--momentum`：动量系数
- `--weight_decay`：L2正则化强度
- `--activation`：激活函数类型

### 9.2 测试模型

```bash
python main.py --mode test --model_path path/to/model
```

### 9.3 超参数搜索

```bash
python main.py --mode search
```

## 10. 项目特点与优势

本项目具有以下特点和优势：

1. **教育价值**：通过手动实现神经网络的各个组件，深入理解神经网络的工作原理和训练过程。

2. **模块化设计**：代码结构清晰，各个组件（模型、激活函数、损失函数、优化器等）都是独立的模块，便于理解和扩展。

3. **灵活性**：支持多种激活函数、优化策略和学习率调度方法，可以根据需要进行配置。

4. **完整性**：包含了神经网络训练的完整流程，从数据加载、预处理到模型训练、评估和超参数搜索。

5. **数值稳定性**：实现了多种数值稳定性技巧，如Xavier/Glorot初始化、梯度裁剪等，以确保训练的稳定性。

6. **性能优化**：使用NumPy的向量化操作，提高了计算效率。

## 11. 总结

本项目通过手动实现一个三层神经网络分类器，展示了神经网络的基本原理和训练过程。通过理解和实现前向传播、反向传播、各种激活函数、损失函数和优化器，我们可以深入理解深度学习的核心概念。

虽然现代深度学习通常使用高级框架（如TensorFlow或PyTorch），但手动实现神经网络有助于：

1. 理解神经网络的数学原理
2. 掌握梯度下降和反向传播算法
3. 了解深度学习框架的内部工作原理
4. 培养调试和优化神经网络的能力

通过这个项目，我们不仅实现了一个功能完整的神经网络分类器，还深入理解了深度学习的基本原理，为进一步学习和应用深度学习奠定了坚实的基础。